#   Optics Clustering

In [1]:
#IMPORTS
import os
import sys
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import OPTICS
from statsmodels.tsa.stattools import coint, grangercausalitytests
import warnings
import itertools
import matplotlib.gridspec as gridspec
import statsmodels.api as sm
warnings.filterwarnings("ignore")

# Add project root to path (handles both CWD=project_root and CWD=research/)
sys.path.insert(0, os.path.abspath('.'))
sys.path.insert(0, os.path.abspath('..'))
from validation.pair_validation import feature_shuffle_permutation_test

# Data Fetch

In [2]:
# Data
stocks = [
    # S&P for Beta
    "^GSPC",
    # Megacap Leaders & Generalists
    "NVDA", "TSM", "AVGO", "AMD", "INTC", "MU", "TXN", "QCOM", "ADI", "MCHP",
    
    # Equipment & Manufacturing
    "ASML", "AMAT", "LRCX", "KLAC", "TER", "ENTG", "NVMI", "TOELY",
    
    # Specialized
    "ON", "NXPI", "STM", "LSCC", "MPWR", "QRVO", "SWKS", "ALAB", "CRDO",
    
    # Intellectual Property & Design Software
    "ARM", "SNPS", "CDNS", "CEVA",
    
    # Memory & Storage
    "WDC", "STX", # Removed extra "MU" here
    
    # Emerging & Mid-Cap
    "GFS", "MRVL", "MTSI", "POWI", "SMTC", "VICR", "CAMT"
]

def fetch_data(stocks):
    data = yf.download(tickers=stocks, period="252d", interval="1h", group_by='ticker', auto_adjust=True, threads=True)
    
    price_series_list = []
    for s in stocks:
        try: 
            if s in data:
                series = data[s]['Close']
                series.name = s
                price_series_list.append(series)
        except Exception as e:
            pass

    if price_series_list:
        df = pd.concat(price_series_list, axis=1)
        # Market-hours-aware fill: only forward-fill within the same trading day
        # to avoid creating false correlations across overnight/weekend gaps
        df = df.groupby(df.index.date).apply(lambda g: g.ffill()).droplevel(0)
        return df
    return pd.DataFrame()

df = fetch_data(stocks)

[                       0%                       ]

[**                     5%                       ]  2 of 41 completed

[*****                 10%                       ]  4 of 41 completed

[******                12%                       ]  5 of 41 completed

[******                12%                       ]  5 of 41 completed

...

[**********************90%******************     ]  37 of 41 completed

[**********************93%********************   ]  38 of 41 completed

[**********************95%*********************  ]  39 of 41 completed

[**********************98%********************** ]  40 of 41 completed

[*********************100%***********************]  41 of 41 completed

## Factor Data Preparation

In [3]:
# ============================================================================
# FEATURE ENGINEERING FOR TRANSIENT REGIME DETECTION
# ============================================================================

# 1. Clean and Prepare Price Data
if isinstance(df.columns, pd.MultiIndex):
    if 'Close' in df.columns.get_level_values(0):
        df = df['Close']
    elif 'Close' in df.columns.get_level_values(1):
        df = df.xs('Close', axis=1, level=1)

# 2. Base Calculations
returns_df = df.pct_change().dropna()
market_returns = returns_df['^GSPC']

# ============================================================================
# CRITICAL CHANGE: Multi-Timeframe Feature Engineering
# ============================================================================

# SHORT-TERM WINDOW (Transient regime detection)
window_short = 50  # ~1 week of hourly data - ALIGNED WITH TRADE DURATION

# MEDIUM-TERM WINDOW (Context/stability check)
window_medium = 147  # ~3 weeks - your original window

print("="*80)
print("FEATURE ENGINEERING - MULTI-TIMEFRAME APPROACH")
print("="*80)
print(f"Short-term window: {window_short} hours (~1 week)")
print(f"Medium-term window: {window_medium} hours (~3 weeks)")
print(f"Optimizing for transient events: 10-50 hour duration\n")


# ============================================================================
# 3A. SHORT-TERM FEATURES (Primary clustering features)
# ============================================================================

print("Calculating SHORT-TERM features (primary regime indicators)...")

# Feature A: SHORT-TERM Volatility (Recent risk behavior)
rolling_vol_short = returns_df.rolling(window=window_short).std()

# Feature B: SHORT-TERM Beta to SPX (Recent market sensitivity)
rolling_cov_mkt_short = returns_df.rolling(window=window_short).cov(market_returns)
rolling_mkt_var_short = market_returns.rolling(window=window_short).var()
rolling_beta_spx_short = rolling_cov_mkt_short.divide(rolling_mkt_var_short, axis=0)

# Feature C: SHORT-TERM Beta to Sector (Recent sector coupling)
non_spx_returns = returns_df.drop(columns=['^GSPC'], errors='ignore').dropna(axis=1, how='all')
sector_sum = non_spx_returns.sum(axis=1)
n_stocks = non_spx_returns.count(axis=1)

# Leave-one-out sector average and rolling beta for each ticker
rolling_beta_sector_short = pd.DataFrame(index=returns_df.index, columns=non_spx_returns.columns)
for ticker in non_spx_returns.columns:
    loo_sector = (sector_sum - non_spx_returns[ticker].fillna(0)) / (n_stocks - 1).clip(lower=1)
    cov = returns_df[ticker].rolling(window=window_short).cov(loo_sector)
    var = loo_sector.rolling(window=window_short).var()
    rolling_beta_sector_short[ticker] = cov / var


# ============================================================================
# 3B. MEDIUM-TERM FEATURES (Context/stability indicators)
# ============================================================================

print("Calculating MEDIUM-TERM features (context indicators)...")

# These help identify if current behavior is unusual vs. longer-term baseline
rolling_vol_medium = returns_df.rolling(window=window_medium).std()

rolling_cov_mkt_medium = returns_df.rolling(window=window_medium).cov(market_returns)
rolling_mkt_var_medium = market_returns.rolling(window=window_medium).var()
rolling_beta_spx_medium = rolling_cov_mkt_medium.divide(rolling_mkt_var_medium, axis=0)

rolling_beta_sector_medium = pd.DataFrame(index=returns_df.index, columns=non_spx_returns.columns)
for ticker in non_spx_returns.columns:
    loo_sector = (sector_sum - non_spx_returns[ticker].fillna(0)) / (n_stocks - 1).clip(lower=1)
    cov = returns_df[ticker].rolling(window=window_medium).cov(loo_sector)
    var = loo_sector.rolling(window=window_medium).var()
    rolling_beta_sector_medium[ticker] = cov / var


# ============================================================================
# 3C. INSTANTANEOUS FEATURES (Momentum/Overbought indicators)
# ============================================================================

print("Calculating INSTANTANEOUS features (momentum indicators)...")

# Feature D: RSI (Momentum/Overextended) - 70 periods for hourly data (~2 weeks)
def calculate_rsi(data, window=70):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

rsi_df = df.apply(calculate_rsi)

# Feature E: Short Term Momentum (5-period return)
momentum_5h = df.pct_change(periods=5)

# Feature F: Momentum Acceleration (change in momentum)
momentum_10h = df.pct_change(periods=10)
momentum_acceleration = momentum_5h - momentum_5h.shift(5)


# ============================================================================
# 3D. REGIME CHANGE INDICATORS (New!)
# ============================================================================

print("Calculating REGIME CHANGE indicators...")

# Detect when short-term behavior diverges from medium-term baseline
# This helps identify when a NEW regime is forming

# Volatility Regime Shift (is vol spiking vs. baseline?)
vol_regime_shift = (rolling_vol_short - rolling_vol_medium) / rolling_vol_medium

# Beta Regime Shift (is market sensitivity changing?)
beta_spx_regime_shift = rolling_beta_spx_short - rolling_beta_spx_medium
beta_sector_regime_shift = rolling_beta_sector_short - rolling_beta_sector_medium


# ============================================================================
# 4. Assemble the Master Time-Series DataFrame (ts_df)
# ============================================================================

print("\nAssembling time-series dataframe...")

ts_data_list = []

for ticker in stocks:
    if ticker == '^GSPC' or ticker not in df.columns: 
        continue
    
    # Extract features for this specific ticker
    temp_df = pd.DataFrame({
        # Price & Returns (baseline)
        'Price': df[ticker],
        'Returns': returns_df[ticker],
        
        # SHORT-TERM FEATURES (Primary clustering features)
        'Vol_Short': rolling_vol_short[ticker],
        'Beta_SPX_Short': rolling_beta_spx_short[ticker],
        'Beta_Sector_Short': rolling_beta_sector_short[ticker],
        
        # MEDIUM-TERM FEATURES (Context)
        'Vol_Medium': rolling_vol_medium[ticker],
        'Beta_SPX_Medium': rolling_beta_spx_medium[ticker],
        'Beta_Sector_Medium': rolling_beta_sector_medium[ticker],
        
        # INSTANTANEOUS FEATURES
        'RSI': rsi_df[ticker],
        'Momentum_5H': momentum_5h[ticker],
        'Momentum_10H': momentum_10h[ticker],
        'Momentum_Accel': momentum_acceleration[ticker],
        
        # REGIME CHANGE INDICATORS (New!)
        'Vol_Regime_Shift': vol_regime_shift[ticker],
        'Beta_SPX_Regime_Shift': beta_spx_regime_shift[ticker],
        'Beta_Sector_Regime_Shift': beta_sector_regime_shift[ticker],
        
    }, index=df.index)
    
    temp_df['Ticker'] = ticker
    ts_data_list.append(temp_df)

if ts_data_list:
    ts_df = pd.concat(ts_data_list).reset_index().set_index(['Datetime', 'Ticker'])
    
    # Drop NaNs created by rolling windows
    initial_rows = len(ts_df)
    ts_df = ts_df.dropna()
    dropped_rows = initial_rows - len(ts_df)
    
    print(f"\n{'='*80}")
    print("TIME-SERIES DATAFRAME CREATED SUCCESSFULLY")
    print(f"{'='*80}")
    print(f"Total rows: {len(ts_df):,}")
    print(f"Rows dropped (NaN): {dropped_rows:,} ({dropped_rows/initial_rows:.1%})")
    print(f"Date range: {ts_df.index.get_level_values('Datetime').min()} to {ts_df.index.get_level_values('Datetime').max()}")
    print(f"Unique tickers: {ts_df.index.get_level_values('Ticker').nunique()}")
    print(f"\nFeature columns: {len([c for c in ts_df.columns if c not in ['Price', 'Returns', 'Ticker']])}")
    print("\nSample data:")
    print(ts_df.head())


# ============================================================================
# 5. OPTIONAL: Static Fundamental DataFrame (Keep or Remove?)
# ============================================================================

# NOTE: For transient regime detection, fundamentals are less relevant
# Transient coupling is driven by events/news, not fundamental similarity
# Consider REMOVING this section unless you plan to use it for filtering

print(f"\n{'='*80}")
print("SKIPPING STATIC FUNDAMENTALS (Not relevant for transient detection)")
print(f"{'='*80}")
print("Transient coupling is driven by events/market dynamics, not fundamental profiles.")
print("If you want to filter pairs by fundamentals later, re-enable this section.\n")

# Uncomment below if you want to keep fundamentals
"""
fundamental_list = []
print("Fetching Static Fundamentals...")

for ticker in stocks:
    if ticker == '^GSPC': continue
    try:
        t = yf.Ticker(ticker)
        info = t.info
        
        fundamental_list.append({
            'Ticker': ticker,
            'Sector': info.get('sector', 'Unknown'),
            'Industry': info.get('industry', 'Unknown'),
            'Market_Cap': info.get('marketCap', np.nan),
        })
    except Exception as e:
        print(f"Could not fetch data for {ticker}: {e}")
        continue

static_df = pd.DataFrame(fundamental_list).set_index('Ticker')
print("Static DataFrame (static_df) Created Successfully!")
"""

print("="*80)
print("FEATURE ENGINEERING COMPLETE - Ready for clustering")
print("="*80)

FEATURE ENGINEERING - MULTI-TIMEFRAME APPROACH
Short-term window: 50 hours (~1 week)
Medium-term window: 147 hours (~3 weeks)
Optimizing for transient events: 10-50 hour duration

Calculating SHORT-TERM features (primary regime indicators)...
Calculating MEDIUM-TERM features (context indicators)...
Calculating INSTANTANEOUS features (momentum indicators)...
Calculating REGIME CHANGE indicators...

Assembling time-series dataframe...

TIME-SERIES DATAFRAME CREATED SUCCESSFULLY
Total rows: 63,143
Rows dropped (NaN): 6,697 (9.6%)
Date range: 2025-03-24 15:30:00+00:00 to 2026-02-23 20:30:00+00:00
Unique tickers: 40

Feature columns: 13

Sample data:
                                       Price   Returns  Vol_Short  \
Datetime                  Ticker                                    
2025-03-24 15:30:00+00:00 NVDA    121.750000 -0.002376   0.010164   
2025-03-24 16:30:00+00:00 NVDA    121.728798 -0.000174   0.010112   
2025-03-24 17:30:00+00:00 NVDA    121.815002  0.000708   0.010080   
2

# Clustering

## Based on an Hourly Time Frame

In [4]:
# ============================================================================
# OPTICS CLUSTERING FOR TRANSIENT REGIME DETECTION
# ============================================================================

if 'ts_df' not in locals():
    raise ValueError("Please run the Feature Engineering cell to create 'ts_df' first.")

# Clean duplicates
ts_df = ts_df[~ts_df.index.duplicated(keep='first')]

# Check Density
density = ts_df.groupby(level='Datetime').size()
valid_timestamps = density[density >= 5].index

print(f"{'='*80}")
print(f"TRANSIENT REGIME DETECTION - OPTICS CLUSTERING")
print(f"{'='*80}")
print(f"Data Density: {len(valid_timestamps)} valid hourly timestamps")
print(f"Date Range: {valid_timestamps.min()} to {valid_timestamps.max()}")

# ============================================================================
# FEATURES: Short-term + regime shift indicators for transient detection
# ============================================================================
features_to_cluster = ['Returns', 'Vol_Short', 'Beta_SPX_Short', 'Beta_Sector_Short', 'RSI', 'Momentum_5H', 'Vol_Regime_Shift', 'Beta_SPX_Regime_Shift', 'Beta_Sector_Regime_Shift']
print(f"\nUsing features: {features_to_cluster}")

# ============================================================================
# CLUSTERING LOOP - Hourly Regime Detection
# ============================================================================

print(f"\n{'='*80}")
print("PHASE 1: Running OPTICS Clustering (Hourly Snapshots)")
print(f"{'='*80}")

cluster_results = []
cluster_quality_log = []

for i, ts in enumerate(valid_timestamps):
    try:
        snapshot = ts_df.xs(ts, level='Datetime')[features_to_cluster].dropna()
        if len(snapshot) < 5: 
            continue
        
        # Scale & PCA (Dimensionality Reduction)
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(snapshot)
        
        pca = PCA(n_components=0.90)
        X_pca = pca.fit_transform(X_scaled)
        
        # OPTICS Clustering
        optics = OPTICS(min_samples=3, metric='euclidean', xi=0.05, min_cluster_size=3)
        optics.fit(X_pca)
        
        # ===== CLUSTER QUALITY VALIDATION (RELAXED) =====
        unique_clusters = len(set(optics.labels_)) - (1 if -1 in optics.labels_ else 0)
        noise_count = (optics.labels_ == -1).sum()
        noise_pct = noise_count / len(optics.labels_)
        total_stocks = len(optics.labels_)
        
        # Quality Metrics
        quality_metrics = {
            'Datetime': ts,
            'Total_Stocks': total_stocks,
            'Unique_Clusters': unique_clusters,
            'Noise_Count': noise_count,
            'Noise_Pct': noise_pct,
            'PCA_Components': X_pca.shape[1],
            'Variance_Explained': pca.explained_variance_ratio_.sum()
        }
        
        # RELAXED Quality Filters
        is_valid = True
        skip_reason = None
        
        if unique_clusters < 1:
            is_valid = False
            skip_reason = "No clusters found (all noise)"
        elif noise_pct > 0.75:  # Relaxed to 75%
            is_valid = False
            skip_reason = f"Too noisy ({noise_pct:.1%} noise)"
        elif unique_clusters > total_stocks * 0.75:
            is_valid = False
            skip_reason = f"Over-fragmented ({unique_clusters} clusters for {total_stocks} stocks)"
        
        quality_metrics['Is_Valid'] = is_valid
        quality_metrics['Skip_Reason'] = skip_reason
        cluster_quality_log.append(quality_metrics)
        
        # Store valid clusters
        if is_valid:
            snapshot['Cluster_ID'] = optics.labels_
            snapshot['Datetime'] = ts
            snapshot['Num_Clusters'] = unique_clusters
            snapshot['Noise_Pct'] = noise_pct
            cluster_results.append(snapshot.reset_index())
        
        # Progress indicator
        if (i + 1) % 100 == 0:
            valid_so_far = len(cluster_results)
            print(f"  Processed {i+1}/{len(valid_timestamps)} timestamps... ({valid_so_far} valid so far)")
            
    except Exception as e:
        cluster_quality_log.append({
            'Datetime': ts,
            'Total_Stocks': np.nan,
            'Unique_Clusters': np.nan,
            'Noise_Count': np.nan,
            'Noise_Pct': np.nan,
            'PCA_Components': np.nan,
            'Variance_Explained': np.nan,
            'Is_Valid': False,
            'Skip_Reason': f"Error: {str(e)[:50]}"
        })
        continue

if not cluster_results:
    raise ValueError("No valid clusters found. Check your data quality and OPTICS parameters.")

cluster_history = pd.concat(cluster_results, ignore_index=True)

print(f"\n{'='*80}")
print("PHASE 1 COMPLETE: Cluster Quality Summary")
print(f"{'='*80}")

# ============================================================================
# CLUSTER QUALITY ANALYSIS
# ============================================================================

df_quality = pd.DataFrame(cluster_quality_log)

total_timestamps = len(df_quality)
valid_timestamps_count = df_quality['Is_Valid'].sum()
invalid_timestamps_count = total_timestamps - valid_timestamps_count

print(f"\nTimestamp Analysis:")
print(f"  Total timestamps processed: {total_timestamps}")
print(f"  Valid clustering windows: {valid_timestamps_count} ({valid_timestamps_count/total_timestamps:.1%})")
print(f"  Invalid/skipped windows: {invalid_timestamps_count} ({invalid_timestamps_count/total_timestamps:.1%})")

if invalid_timestamps_count > 0:
    print(f"\nSkip Reasons:")
    skip_summary = df_quality[~df_quality['Is_Valid']]['Skip_Reason'].value_counts()
    for reason, count in skip_summary.items():
        print(f"  - {reason}: {count} ({count/invalid_timestamps_count:.1%})")

# Valid cluster statistics
valid_quality = df_quality[df_quality['Is_Valid']]
if len(valid_quality) > 0:
    print(f"\nValid Cluster Statistics:")
    print(f"  Avg clusters per timestamp: {valid_quality['Unique_Clusters'].mean():.1f}")
    print(f"  Avg noise percentage: {valid_quality['Noise_Pct'].mean():.1%}")
    print(f"  Avg PCA variance retained: {valid_quality['Variance_Explained'].mean():.1%}")

print(f"\nCluster History Generated:")
print(f"  Total rows: {len(cluster_history)}")
print(f"  Unique timestamps: {cluster_history['Datetime'].nunique()}")
print(f"  Date range: {cluster_history['Datetime'].min()} to {cluster_history['Datetime'].max()}")


# ============================================================================
# PHASE 2: Cluster Stability Analysis
# ============================================================================

print(f"\n{'='*80}")
print("PHASE 2: Analyzing Cluster Stability")
print(f"{'='*80}")

pair_co_cluster_freq = {}

for ts in cluster_history['Datetime'].unique():
    snapshot = cluster_history[cluster_history['Datetime'] == ts]
    
    for cluster_id in snapshot['Cluster_ID'].unique():
        if cluster_id == -1:
            continue
        
        members = snapshot[snapshot['Cluster_ID'] == cluster_id]['Ticker'].tolist()
        
        for s1, s2 in itertools.combinations(sorted(members), 2):
            pair_key = (s1, s2)
            
            if pair_key not in pair_co_cluster_freq:
                pair_co_cluster_freq[pair_key] = 0
            pair_co_cluster_freq[pair_key] += 1

# Calculate frequencies
total_valid_windows = cluster_history['Datetime'].nunique()

pair_stability_data = []
for pair, count in pair_co_cluster_freq.items():
    frequency = count / total_valid_windows
    pair_stability_data.append({
        'Ticker_1': pair[0],
        'Ticker_2': pair[1],
        'Pair': f"{pair[0]}-{pair[1]}",
        'Co_Cluster_Count': count,
        'Co_Cluster_Frequency': frequency,
        'Is_Stable': frequency > 0.25
    })

df_pair_stability = pd.DataFrame(pair_stability_data).sort_values('Co_Cluster_Frequency', ascending=False)

print(f"\nPair Clustering Analysis:")
print(f"  Total unique pairs observed: {len(df_pair_stability)}")
print(f"  Pairs clustered together >50% of time: {(df_pair_stability['Co_Cluster_Frequency'] > 0.50).sum()}")
print(f"  Pairs clustered together >30% of time: {(df_pair_stability['Co_Cluster_Frequency'] > 0.30).sum()}")
print(f"  Pairs clustered together <10% of time: {(df_pair_stability['Co_Cluster_Frequency'] < 0.10).sum()}")

print(f"\n{'='*80}")
print("TOP 15 MOST FREQUENTLY CO-CLUSTERED PAIRS")
print(f"{'='*80}")
print(df_pair_stability[['Pair', 'Co_Cluster_Count', 'Co_Cluster_Frequency']].head(15).to_string(index=False))

print(f"\n{'='*80}")
print("TOP 15 MOST TRANSIENT PAIRS (Rare Co-Clustering)")
print(f"{'='*80}")
transient_pairs = df_pair_stability[
    (df_pair_stability['Co_Cluster_Frequency'] > 0.05) &
    (df_pair_stability['Co_Cluster_Frequency'] < 0.20)
].sort_values('Co_Cluster_Frequency', ascending=True)
print(transient_pairs[['Pair', 'Co_Cluster_Count', 'Co_Cluster_Frequency']].head(15).to_string(index=False))


# ============================================================================
# PHASE 3: Temporal Analysis
# ============================================================================

print(f"\n{'='*80}")
print("PHASE 3: Temporal Analysis - When Do Regimes Shift?")
print(f"{'='*80}")

cluster_history['Date'] = pd.to_datetime(cluster_history['Datetime']).dt.date

daily_cluster_stats = cluster_history.groupby('Date').agg({
    'Cluster_ID': lambda x: len(set(x)) - (1 if -1 in x.values else 0),
    'Ticker': 'count'
}).rename(columns={'Cluster_ID': 'Num_Clusters', 'Ticker': 'Total_Obs'})

print(f"\nDaily Clustering Variability:")
print(f"  Days with high differentiation (>4 clusters): {(daily_cluster_stats['Num_Clusters'] > 4).sum()}")
print(f"  Days with low differentiation (≤2 clusters): {(daily_cluster_stats['Num_Clusters'] <= 2).sum()}")

mean_clusters = daily_cluster_stats['Num_Clusters'].mean()
std_clusters = daily_cluster_stats['Num_Clusters'].std()
regime_shift_days = daily_cluster_stats[
    abs(daily_cluster_stats['Num_Clusters'] - mean_clusters) > 1.5 * std_clusters
]

if len(regime_shift_days) > 0:
    print(f"\nPotential Regime Shift Days (unusual cluster patterns):")
    print(f"  {len(regime_shift_days)} days detected")
    print(f"\nTop 5 Most Unusual Days:")
    top_shifts = regime_shift_days.nlargest(5, 'Num_Clusters')
    for date, row in top_shifts.iterrows():
        print(f"  {date}: {row['Num_Clusters']:.0f} clusters (avg: {mean_clusters:.1f})")

print(f"\n{'='*80}")
print("CLUSTERING PHASE COMPLETE")
print(f"{'='*80}")
print(f"\nData structures created:")
print(f"  - cluster_history: {len(cluster_history)} rows")
print(f"  - df_quality: {len(df_quality)} rows")
print(f"  - df_pair_stability: {len(df_pair_stability)} rows")
print(f"\nReady for pair testing phase")

TRANSIENT REGIME DETECTION - OPTICS CLUSTERING
Data Density: 1579 valid hourly timestamps
Date Range: 2025-03-24 15:30:00+00:00 to 2026-02-23 20:30:00+00:00

Using features: ['Returns', 'Vol_Short', 'Beta_SPX_Short', 'Beta_Sector_Short', 'RSI', 'Momentum_5H', 'Vol_Regime_Shift', 'Beta_SPX_Regime_Shift', 'Beta_Sector_Regime_Shift']

PHASE 1: Running OPTICS Clustering (Hourly Snapshots)


  Processed 100/1579 timestamps... (91 valid so far)


  Processed 200/1579 timestamps... (179 valid so far)


  Processed 300/1579 timestamps... (260 valid so far)


  Processed 400/1579 timestamps... (338 valid so far)


...

  Processed 1300/1579 timestamps... (1122 valid so far)


  Processed 1400/1579 timestamps... (1214 valid so far)


  Processed 1500/1579 timestamps... (1307 valid so far)



PHASE 1 COMPLETE: Cluster Quality Summary

Timestamp Analysis:
  Total timestamps processed: 1579
  Valid clustering windows: 1364 (86.4%)
  Invalid/skipped windows: 215 (13.6%)

Skip Reasons:
  - Too noisy (77.5% noise): 65 (30.2%)
  - Too noisy (80.0% noise): 52 (24.2%)
  - Too noisy (82.5% noise): 45 (20.9%)
  - Too noisy (85.0% noise): 23 (10.7%)
  - Too noisy (90.0% noise): 11 (5.1%)
  - Too noisy (92.5% noise): 10 (4.7%)
  - Too noisy (87.5% noise): 5 (2.3%)
  - Too noisy (76.9% noise): 3 (1.4%)
  - Too noisy (84.6% noise): 1 (0.5%)

Valid Cluster Statistics:
  Avg clusters per timestamp: 3.4
  Avg noise percentage: 58.5%
  Avg PCA variance retained: 92.8%

Cluster History Generated:
  Total rows: 54547
  Unique timestamps: 1364
  Date range: 2025-03-24 16:30:00+00:00 to 2026-02-23 20:30:00+00:00

PHASE 2: Analyzing Cluster Stability



Pair Clustering Analysis:
  Total unique pairs observed: 776
  Pairs clustered together >50% of time: 0
  Pairs clustered together >30% of time: 4
  Pairs clustered together <10% of time: 685

TOP 15 MOST FREQUENTLY CO-CLUSTERED PAIRS
     Pair  Co_Cluster_Count  Co_Cluster_Frequency
QRVO-SWKS               522              0.382698
KLAC-LRCX               497              0.364370
AMAT-LRCX               493              0.361437
AMAT-KLAC               445              0.326246
 ADI-NXPI               357              0.261730
  ADI-TXN               350              0.256598
MCHP-NXPI               317              0.232405
QCOM-QRVO               310              0.227273
 ADI-SWKS               303              0.222141
QCOM-SWKS               292              0.214076
 NXPI-STM               283              0.207478
 NXPI-TXN               277              0.203079
  MCHP-ON               275              0.201613
CDNS-SNPS               267              0.195748
 ADI-QCOM     

# Cluster Formation & Duration Analysis

In [5]:
# ============================================================================
# CLUSTER FORMATION & DISSOLUTION EVENT DETECTION
# ============================================================================
# Instead of just counting co-clustering frequency, detect the MOMENTS when
# pairs START and STOP co-clustering. These formation events are what we
# want to predict.

print(f"{'='*80}")
print("PHASE 2B: CLUSTER FORMATION & DISSOLUTION EVENTS")
print(f"{'='*80}")

# ============================================================================
# Step 1: Build a per-pair, per-timestamp co-clustering indicator
# ============================================================================

# Get all unique pairs that ever co-clustered (from Phase 2)
all_pairs = list(pair_co_cluster_freq.keys())
all_timestamps = sorted(cluster_history['Datetime'].unique())

print(f"Tracking {len(all_pairs)} pairs across {len(all_timestamps)} timestamps...")

# Build co-clustering matrix: for each pair, 1 if co-clustered at that timestamp, 0 otherwise
pair_coclustering = {}

for ts in all_timestamps:
    snapshot = cluster_history[cluster_history['Datetime'] == ts]
    coclustered_at_ts = set()
    
    for cluster_id in snapshot['Cluster_ID'].unique():
        if cluster_id == -1:
            continue
        members = sorted(snapshot[snapshot['Cluster_ID'] == cluster_id]['Ticker'].tolist())
        for s1, s2 in itertools.combinations(members, 2):
            coclustered_at_ts.add((s1, s2))
    
    for pair in all_pairs:
        if pair not in pair_coclustering:
            pair_coclustering[pair] = []
        pair_coclustering[pair].append(1 if pair in coclustered_at_ts else 0)

# ============================================================================
# Step 2: Detect Formation and Dissolution Events
# ============================================================================
# A formation event occurs when a pair transitions from NOT co-clustering
# to co-clustering (0 -> 1 transition).
# A dissolution event is the reverse (1 -> 0 transition).
# We require a minimum gap (MIN_GAP_HOURS) of non-co-clustering before a
# new formation to avoid counting brief flickers.

MIN_GAP_HOURS = 5  # Minimum hours of non-co-clustering before counting a new formation

formation_events = []
dissolution_events = []
cluster_durations = []

for pair, series in pair_coclustering.items():
    ts_series = list(zip(all_timestamps, series))
    
    in_cluster = False
    formation_ts = None
    formation_idx = None
    gap_count = MIN_GAP_HOURS  # Start with full gap so first co-clustering counts as formation
    
    for i, (ts, val) in enumerate(ts_series):
        if val == 1:
            if not in_cluster and gap_count >= MIN_GAP_HOURS:
                # Formation event: pair just started co-clustering after a sufficient gap
                in_cluster = True
                formation_ts = ts
                formation_idx = i
                formation_events.append({
                    'Ticker_1': pair[0],
                    'Ticker_2': pair[1],
                    'Pair': f"{pair[0]}-{pair[1]}",
                    'Formation_Time': ts,
                    'Timestamp_Index': i
                })
            elif not in_cluster:
                # Still in the gap period, but co-clustering again
                in_cluster = True
                formation_ts = ts
                formation_idx = i
            gap_count = 0
        else:
            if in_cluster:
                gap_count += 1
                if gap_count >= MIN_GAP_HOURS:
                    # Dissolution confirmed after sufficient gap
                    # The last co-clustering timestamp was at index (i - gap_count)
                    last_cocluster_idx = i - gap_count
                    dissolution_ts = all_timestamps[min(last_cocluster_idx + 1, len(all_timestamps) - 1)]
                    duration_hours = (all_timestamps[last_cocluster_idx] - all_timestamps[formation_idx]).total_seconds() / 3600.0 if formation_idx is not None else 1.0
                    duration = max(1.0, duration_hours)
                    
                    dissolution_events.append({
                        'Ticker_1': pair[0],
                        'Ticker_2': pair[1],
                        'Pair': f"{pair[0]}-{pair[1]}",
                        'Dissolution_Time': dissolution_ts,
                        'Duration_Hours': duration
                    })
                    
                    if formation_ts is not None:
                        cluster_durations.append({
                            'Ticker_1': pair[0],
                            'Ticker_2': pair[1],
                            'Pair': f"{pair[0]}-{pair[1]}",
                            'Formation_Time': formation_ts,
                            'Dissolution_Time': dissolution_ts,
                            'Duration_Hours': duration
                        })
                    
                    in_cluster = False
                    formation_ts = None
                    formation_idx = None
            else:
                gap_count += 1

    # Handle pairs still in cluster at end of series
    if in_cluster and formation_ts is not None:
        last_cocluster_idx = len(ts_series) - 1
        for j in range(len(ts_series) - 1, -1, -1):
            if ts_series[j][1] == 1:
                last_cocluster_idx = j
                break
        duration_hours = (all_timestamps[last_cocluster_idx] - all_timestamps[formation_idx]).total_seconds() / 3600.0 if formation_idx is not None else 1.0
        duration = max(1.0, duration_hours)
        dissolution_ts = all_timestamps[min(last_cocluster_idx + 1, len(all_timestamps) - 1)]
        
        dissolution_events.append({
            'Ticker_1': pair[0],
            'Ticker_2': pair[1],
            'Pair': f"{pair[0]}-{pair[1]}",
            'Dissolution_Time': dissolution_ts,
            'Duration_Hours': duration
        })
        
        cluster_durations.append({
            'Ticker_1': pair[0],
            'Ticker_2': pair[1],
            'Pair': f"{pair[0]}-{pair[1]}",
            'Formation_Time': formation_ts,
            'Dissolution_Time': dissolution_ts,
            'Duration_Hours': duration
        })

df_formations = pd.DataFrame(formation_events)
df_dissolutions = pd.DataFrame(dissolution_events)
df_durations = pd.DataFrame(cluster_durations)

print(f"\nFormation events detected: {len(df_formations)}")
print(f"Dissolution events detected: {len(df_dissolutions)}")
print(f"Complete cluster episodes (formation + dissolution): {len(df_durations)}")

# ============================================================================
# Step 3: Duration Analysis
# ============================================================================

if len(df_durations) > 0:
    print(f"\n{'='*80}")
    print("CLUSTER DURATION ANALYSIS")
    print(f"{'='*80}")
    
    print(f"\nDuration Statistics (hours):")
    print(f"  Mean:   {df_durations['Duration_Hours'].mean():.1f}")
    print(f"  Median: {df_durations['Duration_Hours'].median():.1f}")
    print(f"  Min:    {df_durations['Duration_Hours'].min():.0f}")
    print(f"  Max:    {df_durations['Duration_Hours'].max():.0f}")
    print(f"  Std:    {df_durations['Duration_Hours'].std():.1f}")
    
    # Duration buckets
    short_lived = len(df_durations[df_durations['Duration_Hours'] <= 10])
    medium_lived = len(df_durations[(df_durations['Duration_Hours'] > 10) & (df_durations['Duration_Hours'] <= 50)])
    long_lived = len(df_durations[df_durations['Duration_Hours'] > 50])
    
    print(f"\nDuration Buckets:")
    print(f"  Short-lived (<=10h):  {short_lived} ({short_lived/len(df_durations):.1%})")
    print(f"  Medium (10-50h):      {medium_lived} ({medium_lived/len(df_durations):.1%})")
    print(f"  Long-lived (>50h):    {long_lived} ({long_lived/len(df_durations):.1%})")

# ============================================================================
# Step 4: Classify pairs
# ============================================================================

print(f"\n{'='*80}")
print("PAIR CLASSIFICATION")
print(f"{'='*80}")

# Transient pairs: multiple formation events with short durations
pair_formation_counts = df_formations.groupby('Pair').size().reset_index(name='Formation_Count')
pair_avg_duration = df_durations.groupby('Pair')['Duration_Hours'].mean().reset_index(name='Avg_Duration')

pair_classification = pair_formation_counts.merge(pair_avg_duration, on='Pair', how='left')
pair_classification = pair_classification.merge(
    df_pair_stability[['Pair', 'Co_Cluster_Frequency']], on='Pair', how='left'
)

# Classify
pair_classification['Category'] = 'unknown'

# Transient: multiple short-lived formations (the target use case)
transient_mask = (pair_classification['Formation_Count'] >= 3) & (pair_classification['Avg_Duration'] <= 30)
pair_classification.loc[transient_mask, 'Category'] = 'transient'

# Stable: high co-clustering frequency OR very long average duration
stable_mask = ((pair_classification['Co_Cluster_Frequency'] > 0.25) | (pair_classification['Avg_Duration'] > 100)) & (pair_classification['Category'] == 'unknown')
pair_classification.loc[stable_mask, 'Category'] = 'stable_candidate'

# Sporadic: few formation events
sporadic_mask = (pair_classification['Formation_Count'] <= 2) & (pair_classification['Category'] == 'unknown')
pair_classification.loc[sporadic_mask, 'Category'] = 'sporadic'

for cat in ['transient', 'stable_candidate', 'sporadic', 'unknown']:
    count = len(pair_classification[pair_classification['Category'] == cat])
    print(f"  {cat:20s}: {count} pairs")

# Show top transient pairs
transient_pairs_classified = pair_classification[pair_classification['Category'] == 'transient'].sort_values('Formation_Count', ascending=False)
if len(transient_pairs_classified) > 0:
    print(f"\nTop 15 Transient Pairs (target for prediction):")
    print(transient_pairs_classified[['Pair', 'Formation_Count', 'Avg_Duration', 'Co_Cluster_Frequency']].head(15).to_string(index=False))

# Show stable/cointegration candidates
stable_candidates = pair_classification[pair_classification['Category'] == 'stable_candidate'].sort_values('Co_Cluster_Frequency', ascending=False)
if len(stable_candidates) > 0:
    print(f"\nStable/Cointegration Candidates (consistently co-clustered):")
    print(stable_candidates[['Pair', 'Formation_Count', 'Avg_Duration', 'Co_Cluster_Frequency']].head(10).to_string(index=False))

print(f"\n{'='*80}")
print("FORMATION/DISSOLUTION ANALYSIS COMPLETE")
print(f"{'='*80}")
print(f"\nData structures created:")
print(f"  - df_formations: {len(df_formations)} formation events")
print(f"  - df_dissolutions: {len(df_dissolutions)} dissolution events")
print(f"  - df_durations: {len(df_durations)} complete cluster episodes")
print(f"  - pair_classification: {len(pair_classification)} pairs classified")
print(f"\nUse df_formations for validation (these are the events to test)")
print(f"Use transient pairs for prediction modeling")


# ============================================================================
# FIX 5: Attach duration metadata (without filtering by future-known duration)
# ============================================================================
# Duration_Hours is kept as metadata for analysis but NOT used to filter,
# because duration is only known after the episode ends (look-ahead bias).

MIN_EPISODE_HOURS = 5

if len(df_durations) > 0:
    df_formations_actionable = df_formations.merge(
        df_durations[['Pair', 'Formation_Time', 'Duration_Hours']],
        on=['Pair', 'Formation_Time'], how='left'
    )
    
    print(f"\n{'='*80}")
    print("ACTIONABLE FORMATION EVENTS (FIX 5 + FIX 13)")
    print(f"{'='*80}")
    print(f"  All formation events:        {len(df_formations)}")
    print(f"  With duration metadata:      {df_formations_actionable['Duration_Hours'].notna().sum()}")
    print(f"  NOTE: Duration_Hours kept as metadata only (no look-ahead filter)")
    
    for bucket_h in [5, 10, 20]:
        n = len(df_formations_actionable[df_formations_actionable['Duration_Hours'] >= bucket_h])
        print(f"  With duration >= {bucket_h}h:        {n}")
else:
    df_formations_actionable = df_formations.copy()
    print("WARNING: No duration data; using all formations as actionable.")

PHASE 2B: CLUSTER FORMATION & DISSOLUTION EVENTS
Tracking 776 pairs across 1364 timestamps...



Formation events detected: 20242
Dissolution events detected: 20242
Complete cluster episodes (formation + dissolution): 20242

CLUSTER DURATION ANALYSIS

Duration Statistics (hours):
  Mean:   19.9
  Median: 1.0
  Min:    1
  Max:    1178
  Std:    44.2

Duration Buckets:
  Short-lived (<=10h):  14156 (69.9%)
  Medium (10-50h):      3615 (17.9%)
  Long-lived (>50h):    2471 (12.2%)

PAIR CLASSIFICATION
  transient           : 681 pairs
  stable_candidate    : 6 pairs
  sporadic            : 11 pairs
  unknown             : 78 pairs

Top 15 Transient Pairs (target for prediction):
     Pair  Formation_Count  Avg_Duration  Co_Cluster_Frequency
  STX-WDC               75     22.506667              0.177419
NXPI-POWI               62     27.403226              0.178886
 ADI-POWI               56     22.964286              0.124633
NXPI-QRVO               55     28.436364              0.140762
 POWI-STM               54     24.518519              0.134897
ASML-LRCX               53     24

# Correctness Checks

In [6]:
# ============================================================================
# CORRECTNESS CHECKS: Feature-Shuffle Permutation Test, OOS Split, Sensitivity
# ============================================================================

# ============================================================================
# 1. FEATURE-SHUFFLE PERMUTATION TEST (FIX 1)
# ============================================================================
# The old test shuffled cluster *labels*, which preserves cluster sizes and
# therefore preserves co-clustering rates by construction (tautological).
#
# The correct approach: shuffle the *feature vectors* across tickers at each
# timestamp, then re-run the full StandardScaler -> PCA -> OPTICS pipeline.
# This breaks the ticker-feature mapping while preserving the joint
# distribution of features, providing a proper null hypothesis.

print(f"{'='*80}")
print("CORRECTNESS CHECK 1: Feature-Shuffle Permutation Test (FIX 1)")
print(f"{'='*80}")

optics_params = dict(min_samples=3, metric='euclidean', xi=0.05, min_cluster_size=3)
features_to_cluster = [
    'Returns', 'Vol_Short', 'Beta_SPX_Short', 'Beta_Sector_Short',
    'RSI', 'Momentum_5H', 'Vol_Regime_Shift', 'Beta_SPX_Regime_Shift',
    'Beta_Sector_Regime_Shift'
]

perm_result = feature_shuffle_permutation_test(
    ts_df=ts_df,
    features_to_cluster=features_to_cluster,
    optics_params=optics_params,
    pair_co_cluster_freq=pair_co_cluster_freq,
    total_valid_windows=cluster_history['Datetime'].nunique(),
    n_permutations=30,
    n_sample_timestamps=80,
)

print(f"\nFraction of pairs significant at p<0.05: {perm_result['fraction_significant']:.1%}")

# Show top pairs by Z-score
top_z = sorted(perm_result['pair_zscores'].items(), key=lambda x: x[1], reverse=True)[:15]
print(f"\nTop 15 pairs by permutation Z-score:")
for pair, z in top_z:
    print(f"  {pair[0]:6s}-{pair[1]:6s}  Z = {z:.2f}")

# ============================================================================
# 2. OUT-OF-SAMPLE SPLIT
# ============================================================================

print(f"\n{'='*80}")
print("CORRECTNESS CHECK 2: Out-of-Sample Split")
print(f"{'='*80}")

all_dates = sorted(cluster_history['Datetime'].unique())
split_point = int(len(all_dates) * 0.67)
split_timestamp = all_dates[split_point]

train_history = cluster_history[cluster_history['Datetime'] <= split_timestamp]
test_history = cluster_history[cluster_history['Datetime'] > split_timestamp]

print(f"\nSplit point: {split_timestamp}")
print(f"Training period: {all_dates[0]} to {split_timestamp} ({len(train_history)} rows)")
print(f"Test period: {split_timestamp} to {all_dates[-1]} ({len(test_history)} rows)")

def calc_pair_freq(history_df):
    freq = {}
    total = history_df['Datetime'].nunique()
    for ts in history_df['Datetime'].unique():
        snap = history_df[history_df['Datetime'] == ts]
        for cid in snap['Cluster_ID'].unique():
            if cid == -1:
                continue
            members = sorted(snap[snap['Cluster_ID'] == cid]['Ticker'].tolist())
            for s1, s2 in itertools.combinations(members, 2):
                freq[(s1, s2)] = freq.get((s1, s2), 0) + 1
    return {k: v / total for k, v in freq.items()}

train_freq = calc_pair_freq(train_history)
test_freq = calc_pair_freq(test_history)

common_pairs = set(train_freq.keys()) & set(test_freq.keys())
if len(common_pairs) > 0:
    train_vals = [train_freq[p] for p in common_pairs]
    test_vals = [test_freq[p] for p in common_pairs]
    from scipy.stats import pearsonr
    oos_corr, oos_pval = pearsonr(train_vals, test_vals)
    
    print(f"\nPairs observed in both periods: {len(common_pairs)}")
    print(f"Correlation of co-clustering frequency (train vs test): {oos_corr:.3f} (p={oos_pval:.4f})")
    
    if oos_corr > 0.5:
        print("RESULT: Good out-of-sample stability")
    elif oos_corr > 0.2:
        print("RESULT: Moderate out-of-sample stability")
    else:
        print("WARNING: Poor out-of-sample stability")
else:
    print("WARNING: No common pairs between train and test periods")

oos_split_timestamp = split_timestamp

# ============================================================================
# 3. OPTICS PARAMETER SENSITIVITY CHECK
# ============================================================================

print(f"\n{'='*80}")
print("CORRECTNESS CHECK 3: OPTICS Parameter Sensitivity")
print(f"{'='*80}")

param_configs = [
    {'min_samples': 2, 'xi': 0.05, 'min_cluster_size': 2},
    {'min_samples': 3, 'xi': 0.05, 'min_cluster_size': 3},
    {'min_samples': 3, 'xi': 0.03, 'min_cluster_size': 3},
    {'min_samples': 5, 'xi': 0.05, 'min_cluster_size': 5},
    {'min_samples': 3, 'xi': 0.10, 'min_cluster_size': 3},
]

sample_timestamps_sens = valid_timestamps[::max(1, len(valid_timestamps) // 50)][:50]
print(f"Testing {len(param_configs)} configs on {len(sample_timestamps_sens)} timestamps...\n")

sensitivity_results = []
for config in param_configs:
    config_clusters = []
    config_noise = []
    for ts in sample_timestamps_sens:
        try:
            snapshot = ts_df.xs(ts, level='Datetime')[features_to_cluster].dropna()
            if len(snapshot) < 5:
                continue
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(snapshot)
            pca = PCA(n_components=0.90)
            X_pca = pca.fit_transform(X_scaled)
            optics = OPTICS(min_samples=config['min_samples'], metric='euclidean',
                            xi=config['xi'], min_cluster_size=config['min_cluster_size'])
            optics.fit(X_pca)
            n_clusters = len(set(optics.labels_)) - (1 if -1 in optics.labels_ else 0)
            noise_pct = (optics.labels_ == -1).sum() / len(optics.labels_)
            config_clusters.append(n_clusters)
            config_noise.append(noise_pct)
        except Exception:
            continue
    if config_clusters:
        sensitivity_results.append({
            'min_samples': config['min_samples'],
            'xi': config['xi'],
            'min_cluster_size': config['min_cluster_size'],
            'avg_clusters': np.mean(config_clusters),
            'avg_noise_pct': np.mean(config_noise),
            'std_clusters': np.std(config_clusters)
        })

df_sensitivity = pd.DataFrame(sensitivity_results)
print("Parameter Sensitivity Results:")
print(df_sensitivity.to_string(index=False))

print(f"\n{'='*80}")
print("ALL CORRECTNESS CHECKS COMPLETE")
print(f"{'='*80}")


CORRECTNESS CHECK 1: Feature-Shuffle Permutation Test (FIX 1)



Fraction of pairs significant at p<0.05: 13.5%

Top 15 pairs by permutation Z-score:
  KLAC  -LRCX    Z = 16.40
  AMAT  -KLAC    Z = 15.61
  QRVO  -SWKS    Z = 14.18
  AMAT  -LRCX    Z = 12.70
  MCHP  -POWI    Z = 8.97
  KLAC  -TSM     Z = 8.84
  ADI   -SWKS    Z = 7.84
  POWI  -SWKS    Z = 7.47
  ASML  -LRCX    Z = 7.37
  AMAT  -ASML    Z = 6.98
  NXPI  -STM     Z = 6.96
  ADI   -NXPI    Z = 6.85
  QCOM  -SWKS    Z = 6.83
  MCHP  -ON      Z = 6.65
  NXPI  -POWI    Z = 6.47

CORRECTNESS CHECK 2: Out-of-Sample Split

Split point: 2025-10-31 13:30:00+00:00
Training period: 2025-03-24 16:30:00+00:00 to 2025-10-31 13:30:00+00:00 (36551 rows)
Test period: 2025-10-31 13:30:00+00:00 to 2026-02-23 20:30:00+00:00 (17996 rows)



Pairs observed in both periods: 747
Correlation of co-clustering frequency (train vs test): 0.665 (p=0.0000)
RESULT: Good out-of-sample stability

CORRECTNESS CHECK 3: OPTICS Parameter Sensitivity
Testing 5 configs on 50 timestamps...



Parameter Sensitivity Results:
 min_samples   xi  min_cluster_size  avg_clusters  avg_noise_pct  std_clusters
           2 0.05                 2          8.44         0.4035      1.733897
           3 0.05                 3          3.30         0.5970      1.187434
           3 0.03                 3          3.60         0.5405      1.077033
           5 0.05                 5          1.30         0.6570      0.500000
           3 0.10                 3          2.50         0.6840      0.964365

ALL CORRECTNESS CHECKS COMPLETE


# Save Artifacts

In [7]:
# ============================================================================
# SAVE ARTIFACTS FOR SIGNALS NOTEBOOK
# ============================================================================

import os
import pickle

data_dir = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'data')
os.makedirs(data_dir, exist_ok=True)

save_items = {
    'ts_df': ts_df,
    'df_formations': df_formations,
    'df_formations_actionable': df_formations_actionable,
    'pair_classification': pair_classification,
    'cluster_history': cluster_history,
    'df_durations': df_durations,
    'df_pair_stability': df_pair_stability,
    'oos_split_timestamp': oos_split_timestamp,
    'pair_co_cluster_freq': pair_co_cluster_freq,
}

for name, obj in save_items.items():
    path = os.path.join(data_dir, f'{name}.pkl')
    with open(path, 'wb') as f:
        pickle.dump(obj, f)
    size = len(obj) if hasattr(obj, '__len__') else 1
    print(f'Saved {name} -> {path} ({size} items)')

print(f'\nAll artifacts saved to {data_dir}/')
print('optics-signals.ipynb can now load these files.')


Saved ts_df -> /Users/jack/Documents/code/pairs-test/research/data/ts_df.pkl (63143 items)
Saved df_formations -> /Users/jack/Documents/code/pairs-test/research/data/df_formations.pkl (20242 items)
Saved df_formations_actionable -> /Users/jack/Documents/code/pairs-test/research/data/df_formations_actionable.pkl (20242 items)
Saved pair_classification -> /Users/jack/Documents/code/pairs-test/research/data/pair_classification.pkl (776 items)
Saved cluster_history -> /Users/jack/Documents/code/pairs-test/research/data/cluster_history.pkl (54547 items)
Saved df_durations -> /Users/jack/Documents/code/pairs-test/research/data/df_durations.pkl (20242 items)
Saved df_pair_stability -> /Users/jack/Documents/code/pairs-test/research/data/df_pair_stability.pkl (776 items)
Saved oos_split_timestamp -> /Users/jack/Documents/code/pairs-test/research/data/oos_split_timestamp.pkl (1 items)


Saved pair_co_cluster_freq -> /Users/jack/Documents/code/pairs-test/research/data/pair_co_cluster_freq.pkl (776 items)

All artifacts saved to /Users/jack/Documents/code/pairs-test/research/data/
optics-signals.ipynb can now load these files.
